In [2]:
import numpy as np
import pandas as pd
import sys
import os
import duckdb
sys.path.append('..')
from helper import *
setwd()

Objects = Objects()

In [5]:
import geopy.geocoders
from geopy.geocoders import Nominatim
geopy.geocoders.options.default_user_agent = 'app_adsdb'

#geolocator = Nominatim(user_agent='app_adsdb/1')
geolocator = Nominatim(timeout=3)

In [36]:
con = duckdb.connect(database="data/trusted/db_housing.db".format(id), read_only=True)
df_housing = con.execute("SELECT * FROM housing").fetchdf()
con.close()

con = duckdb.connect(database="data/trusted/db_hospitals.db".format(id), read_only=True)
df_hospitals = con.execute("SELECT * FROM hospitals").fetchdf()
con.close()

con = duckdb.connect(database="data/trusted/db_schools.db".format(id), read_only=True)
df_schools = con.execute("SELECT * FROM schools").fetchdf()
con.close()

In [ ]:

Lat = np.array(df_housing[["lat"]]).flatten()
Long = np.array(df_housing[["long"]]).flatten()

lat1 = Lat[1]
long1 = Long[1]

duckdb.query("""

                SELECT
                    count(*), (
                    6371 * acos ( cos ( radians({}) )
                    * cos( radians( h.X ) )
                    * cos( radians( h.Y ) - radians({}) )
                    + sin ( radians({}) )
                    * sin( radians( h.X ) )
                    )
                ) AS distance
                FROM df_hospitals h
                WHERE distance < 10
                


                """.format(lat1,long1,lat1)).to_df()


In [43]:
duckdb.query("""
                SELECT hosp.X, hosp.Y, hous.lat, hous.long, SQRT((hosp.X - hous.lat)*(hosp.X - hous.lat) + (hosp.Y - hous.long)*(hosp.Y - hous.long))
                FROM df_hospitals hosp, df_housing hous
                WHERE (hosp.X - hous.lat)*(hosp.X - hous.lat) + (hosp.Y - hous.long)*(hosp.Y - hous.long) < 150*150
                ORDER BY (hosp.X - hous.lat)*(hosp.X - hous.lat) + (hosp.Y - hous.long)*(hosp.Y - hous.long)
                """.format(lat1,long1,lat1)).to_df()

,X,Y,lat,long,"sqrt((((hosp.""X"") - (hous.lat)) * ((hosp.""X"") - (hous.lat))) + (((hosp.""Y"") - (hous.long)) * ((hosp.""Y"") - (hous.long))))"
0,-64.751478,17.733195,2.83091,-2.83100,70.641810
1,-64.751478,17.733195,2.83091,-2.83100,70.641810
2,-64.751478,17.733195,2.93300,-2.93308,70.769227
3,-64.751478,17.733195,2.93300,-2.93308,70.769227
4,-64.914717,18.340605,2.83091,-2.83100,70.976805
...,...,...,...,...,...
5068974,-80.331523,25.900337,25.76660,-80.13350,149.999955
5068975,-66.474682,18.433910,42.69320,-84.43670,149.999963
5068976,-66.072450,18.391933,30.58520,-96.31330,149.999972
5068977,-64.914717,18.340605,42.75710,-86.09490,149.999983


In [30]:
duckdb.query("""
                SELECT hosp.X, hosp.Y, hous.lat, hous.long, ABS(hosp.X - hous.lat) + ABS(hosp.Y - hous.long)
                FROM df_schools hosp, df_housing hous
                WHERE ABS(hosp.X - hous.lat) + ABS(hosp.Y - hous.long) < 150*150
                ORDER BY ABS(hosp.X - hous.lat) + ABS(hosp.Y - hous.long)
                """.format(lat1,long1,lat1)).to_df()

,X,Y,lat,long,"(abs((hosp.""X"") - (hous.lat))) + (abs((hosp.""Y"") - (hous.long)))"
0,-64.686484,17.744582,40.0210,-82.8079,205.259966
1,-64.715303,17.746411,40.0210,-82.8079,205.290613
2,-64.743407,17.727774,40.0210,-82.8079,205.300081
3,-64.760782,17.725168,40.0210,-82.8079,205.314850
4,-64.737319,17.764167,40.0210,-82.8079,205.330386
...,...,...,...,...,...
1023675,-171.714711,63.777510,41.5622,-81.5685,358.622921
1023676,-171.714711,63.777510,41.4816,-81.7018,358.675621
1023677,-171.714711,63.777510,41.4425,-81.7441,358.678821
1023678,-171.714711,63.777510,41.4425,-81.7441,358.678821


In [42]:
con = duckdb.connect(database="data/trusted/db_housing.db".format(id), read_only=True)
df_housing = con.execute("SELECT * FROM housing").fetchdf()
con.close()

df_housing = df_housing.head(100000)
df_housing.describe()

,id,price,sqfeet,beds,baths,cats_allowed,dogs_allowed,smoking_allowed,wheelchair_access,electric_vehicle_charge,comes_furnished,lat,long
count,1.000000e+05,1.000000e+05,100000.000000,100000.000000,100000.000000,100000.00000,100000.000000,100000.000000,100000.000000,100000.000000,100000.000000,99512.000000,99512.000000
mean,7.040963e+09,1.452930e+03,1003.311890,1.892920,1.486380,0.73037,0.714460,0.733900,0.083160,0.011950,0.048530,36.824905,-92.864677
std,8.776520e+06,3.583893e+04,2498.781884,0.874426,0.592561,0.44377,0.451673,0.441919,0.276125,0.108661,0.214884,5.678003,16.740685
min,7.004042e+09,0.000000e+00,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,-24.955800,-159.495000
25%,7.035955e+09,8.190000e+02,750.000000,1.000000,1.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,32.709700,-101.961000
50%,7.043284e+09,1.049000e+03,948.000000,2.000000,1.000000,1.00000,1.000000,1.000000,0.000000,0.000000,0.000000,36.548400,-87.474200
75%,7.048400e+09,1.395000e+03,1147.000000,2.000000,2.000000,1.00000,1.000000,1.000000,0.000000,0.000000,0.000000,40.615200,-81.328900
max,7.051292e+09,7.955555e+06,775814.000000,8.000000,6.500000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,102.036000,94.124800


In [8]:
Lat = np.array(df_housing[["lat"]]).flatten()
Long = np.array(df_housing[["long"]]).flatten()

lat1 = Lat[1]
long1 = Long[1]

print(lat1,long1)


41.5543 -81.488


In [8]:
df_hospitals.head()

,X,Y,OBJECTID,ID,NAME,ADDRESS,CITY,STATE,ZIP,ZIP4,...,VAL_DATE,WEBSITE,STATE_ID,ALT_NAME,ST_FIPS,OWNER,TTL_STAFF,BEDS,TRAUMA,HELIPAD
0,-94.945477,29.747620,8497,76777520,HOUSTON METHODIST SAN JACINTO HOSPITAL ALEXAND...,1700 JAMES BOWIE DRIVE,BAYTOWN,TX,77520,NOT AVAILABLE,...,2017-12-18,http://www.houstonmethodist.org/locations/san-...,NOT AVAILABLE,NOT AVAILABLE,48.0,NON-PROFIT,-999.0,182.0,NOT AVAILABLE,Y
1,-101.294085,48.232226,8504,5258701,TRINITY HOSPITALS,1 W BURDICK EXPY,MINOT,ND,58701,NOT AVAILABLE,...,2017-10-02,http://www.trinityhealth.org,5055A,NOT AVAILABLE,38.0,NON-PROFIT,-999.0,251.0,LEVEL II,Y
2,-80.231076,36.418068,8505,10527016,LIFEBRITE COMMUNITY HOSPITAL OF STOKES,1570 NC 8 & 89 HWY NORTH,DANBURY,NC,27016,NOT AVAILABLE,...,2017-11-09,http://lifebritestokes.com/,NOT AVAILABLE,PIONEER COMMUNITY HOSPITAL OF STOKES,37.0,PROPRIETARY,-999.0,93.0,NOT AVAILABLE,NOT AVAILABLE
3,-95.440687,30.014288,8506,77677090,PROVIDENCE HOSPITAL OF NORTH HOUSTON LLC,16750 RED OAK DR,HOUSTON,TX,77090,NOT AVAILABLE,...,2017-12-18,http://phnh.net/,NOT AVAILABLE,NOT AVAILABLE,48.0,PROPRIETARY,-999.0,16.0,NOT AVAILABLE,NOT AVAILABLE
4,-98.225730,26.186992,8507,78178503,MCCALLEN MEDICAL CENTER,301 WEST EXPRESSWAY 83,MCALLEN,TX,78503,NOT AVAILABLE,...,2017-12-18,http://www.mcallenmedicalcenter.com/,NOT AVAILABLE,NOT AVAILABLE,48.0,PROPRIETARY,-999.0,441.0,LEVEL III,Y


In [17]:
df_housing.head()

,id,url,region,region_url,price,type,sqfeet,beds,baths,cats_allowed,...,wheelchair_access,electric_vehicle_charge,comes_furnished,laundry_options,parking_options,image_url,description,lat,long,state
0,7027976195,https://cleveland.craigslist.org/apa/d/clevela...,cleveland,https://cleveland.craigslist.org,785,house,1375,4,1.5,1,...,0,0,0,w/d hookups,detached garage,https://images.craigslist.org/00f0f_6GVrSu2L7s...,BEAUTIFUL 3 BEDROOM HOME WITH 1.5 BATHS LARGE...,41.4416,-81.5486,oh
1,7044703510,https://cleveland.craigslist.org/apa/d/clevela...,cleveland,https://cleveland.craigslist.org,747,apartment,630,1,1.0,1,...,0,0,0,laundry in bldg,off-street parking,https://images.craigslist.org/00B0B_9eGfUAyMvN...,Are you looking for the ultimate in apartment ...,41.5543,-81.4880,oh
2,7044718204,https://cleveland.craigslist.org/apa/d/bedford...,cleveland,https://cleveland.craigslist.org,690,apartment,1000,2,1.5,1,...,1,0,0,laundry in bldg,off-street parking,https://images.craigslist.org/01111_lgdIUBbFrK...,www.columbusparkoh.com Columbus Park Apartmen...,41.3979,-81.5105,oh
3,7044670968,https://cleveland.craigslist.org/apa/d/save-gr...,cleveland,https://cleveland.craigslist.org,842,apartment,750,2,1.0,1,...,0,0,0,laundry in bldg,off-street parking,https://images.craigslist.org/00X0X_5fWPdpMaxT...,Edgewater Landing apartments features complete...,41.4735,-81.7398,oh
4,7044651471,https://cleveland.craigslist.org/apa/d/quiet-s...,cleveland,https://cleveland.craigslist.org,700,apartment,800,2,1.0,0,...,0,0,0,laundry in bldg,off-street parking,https://images.craigslist.org/00T0T_66PYdgMEkW...,Upstairs unit of a quad. 3 bright freshly redo...,41.5622,-81.5685,oh
